## TODOS

- Impute values where #rooms is missing
- Add more features
- Add data exploration

inspo -> https://towardsdatascience.com/web-scraping-apartment-listings-in-stockholm-3fcebacf8be6

In [1]:
# imports
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm as tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [48]:
# Retrieving the hrefs of the links in the page
url = 'https://www.booli.se/slutpriser/stockholm/1?page=1'
html_page = requests.get(url)
soup = BeautifulSoup(html_page.text, 'html.parser')
select = soup.find('div', class_="_2m6km uC2y2 _3oDFL")
links = select.find_all('a')
hrefs = [link.get('href') for link in links]

In [49]:
# function to calculate number of pages that can be scraped
def get_nr_of_pages(url):
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    select = soup.find('div', class_="EuKIv _36W0F")
    nr_of_objects = select.find('span').text.split(' ')[3]
    nr_of_pages = int(nr_of_objects) // 35 + 1
    return nr_of_pages

get_nr_of_pages(url)

5987

In [50]:
# Retrieving the hrefs of the links in the page
nr_of_objects = 4000
hrefs = []

for i in range(1,nr_of_objects//35+2):
    url = 'https://www.booli.se/slutpriser/stockholm/1?page=' + str(i)
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    select = soup.find('div', class_="_2m6km uC2y2 _3oDFL")
    links = select.find_all('a')
    links_in_this_page = [link.get('href') for link in links]
    hrefs.extend(links_in_this_page)
    hrefs = hrefs[0:nr_of_objects]

print('Collected', len(hrefs), 'links from', i, 'pages')

Collected 4000 links from 115 pages


In [51]:
def get_apartment_info(url_suffix):
    url = 'https://www.booli.se' + url_suffix
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    select = soup.find('div', class_="_2epd7 _12LuH")
    infobox_1 = select.find('div', class_="_2epd7")
    infobox_2 = soup.find('div', class_="_2IyrD _36W0F _16dH_")

    # extracting name, price, size and area from left infobox.
    name = infobox_2.find('h1').text
    price = infobox_2.find('h2').text
    size_and_area = infobox_2.findAll('h4')
    size = size_and_area[0].text
    area = size_and_area[1].text

    # print(name)
    # print(price)
    # print(size)
    # print(area)

    # extracting asking price
    try:
        asking_price_div = infobox_1.select('div:contains("Utropspris")')[0]
        asking_price = asking_price_div.find('div', class_="_18w8g").text
        # print(asking_price, '\n')
    except:
        asking_price = 'N/A'
        # print('No asking price')
        # print('\n')

    return [name, price, size, area, asking_price, url]

In [53]:
# Creating a dataframe with the scraped data, ignoring rows without relevant data

df = pd.DataFrame(columns=['name', 'price', 'size', 'area', 'asking_price', 'url'])

for i in tqdm(range(nr_of_objects)):
    try:
        df.loc[i] = get_apartment_info(hrefs[i])
    except:
        print('Error', 'i=', i)
        print(hrefs[i])

  0%|          | 0/4000 [00:00<?, ?it/s]

Error i= 81
/annons/4719043
Error i= 89
/bostad/665243
Error i= 172
/annons/3123851
Error i= 220
/annons/4699652
Error i= 449
/bostad/2219982
Error i= 697
/bostad/3880316
Error i= 705
/bostad/133607
Error i= 768
/annons/3825608
Error i= 770
/annons/4094053
Error i= 842
/bostad/3888372
Error i= 988
/annons/1283967
Error i= 989
/bostad/565121
Error i= 990
/annons/1360677
Error i= 991
/annons/1370114
Error i= 992
/annons/1388368
Error i= 993
/annons/1320082
Error i= 994
/annons/1405455
Error i= 996
/annons/1606984
Error i= 997
/annons/1645173
Error i= 998
/annons/1645192
Error i= 999
/annons/1632401
Error i= 1000
/annons/1706581
Error i= 1001
/annons/1709323
Error i= 1002
/bostad/1917224
Error i= 1003
/bostad/1982320
Error i= 1004
/bostad/1843977
Error i= 1006
/bostad/1982324
Error i= 1007
/bostad/1477571
Error i= 1008
/bostad/2096967
Error i= 1009
/annons/1843716
Error i= 1010
/bostad/2083978
Error i= 1011
/annons/2041806
Error i= 1012
/annons/2052924
Error i= 1013
/bostad/1804453
Error 

In [140]:
# saving the dataframe to a csv file
# df.to_csv('housing_info.csv', index=False)

In [6]:
# reading the csv file
df = pd.read_csv('housing_info.csv')
print('Length of dataframe:', len(df))
df_copy = df.head().copy()

Length of dataframe: 3638


## Clean the data

In [7]:
df

,name,price,size,area,asking_price,rooms,type
0,Jämtlands­gatan 152,2600000,78.0,Hässelby-Vällingby,NaN,3.0,Lägenhet
1,Jämtlands­gatan 97,2800000,66.0,Vällingby Parkstad,2 795 000 kr,2.0,Lägenhet
2,Dala­gatan 86G,3320000,34.0,Vasastan,3 395 000 kr,1.0,Lägenhet
3,Valhalla­vägen 155,6500000,67.0,Östermalm Nedre Gärdet,6 200 000 kr,2.5,Lägenhet
4,Rådmans­gatan 1B,10850000,81.0,Östermalm,11 250 000 kr,3.0,Lägenhet
...,...,...,...,...,...,...,...
3633,Ymsen­vägen 8,5075000,70.0,Årsta,4 495 000 kr,3.0,Lägenhet
3634,Sandfjärds­gatan 98,4370000,77.0,Årsta,3 195 000 kr,4.0,Lägenhet
3635,Möckel­vägen 9,3200000,40.0,Årsta,2 495 000 kr,2.0,Lägenhet
3636,Sandfjärds­gatan 15,2900000,57.5,Årsta,2 595 000 kr,2.0,Lägenhet


In [ ]:
def clean_data(row):
    
    # convert price to int and delete ' kr'
    row['price'] = int(row['price'].replace(' kr', '').replace(' ', ''))

    # split up size into size and #rooms
    size_and_rooms = row['size'].replace('½', '.5').split(',')
    if len(size_and_rooms) > 1:
        row['size'] = float(size_and_rooms[0].split(' ')[0])
        row['rooms'] = float(size_and_rooms[1].split('rum')[0])
    else:
        row['size'] = int(size_and_rooms[0].split(' ')[0])
        row['rooms'] = 'N/A'

    # split up area into area and house type
    area_and_house_type = row['area'].split(',')
    row['area'] = area_and_house_type[1]
    row['type'] = area_and_house_type[0]

    return row

df = df.apply(clean_data, axis=1)

In [13]:
df['price_per_m2'] = df['price'] / df['size']
df['price_per_m2'] = df['price_per_m2'].round(2)

In [21]:
df['area'].unique()

array([' Hässelby-Vällingby', ' Vällingby Parkstad', ' Vasastan',
       ' Östermalm Nedre Gärdet', ' Östermalm', ' Vasastan Birkastan',
       ' Vasastan Norrmalm', ' Norra Djurgården', ' Vasastan Odenplan',
       ' Hornstull', ' Enskede', ' Farsta', ' Sköndal', ' Bagarmossen',
       ' Skärholmen', ' Hornsbergs Strand', ' Kungsholmen', ' Skarpnäck',
       ' Hässelby Strand', ' Liljeholmen', ' Spånga Solhem',
       ' Midsommarkransen', ' Vällingby Centrum', ' Hägersten',
       ' Hässelby Södra Villastad', ' Solberga', ' Årsta', ' Fruängen',
       ' Blåsut', ' Högdalen', ' Bromma', ' Globen', ' Hägerstensåsen',
       ' Norra Djurgårdsstaden', ' Södermalm Högalid', ' Södermalm',
       ' Johanneshov Gullmarsplan', ' Aspudden', ' Hässelby',
       ' Hökarängen', ' Kärrtorp', ' Annedal', ' Liljeholmskajen',
       ' Älvsjö', ' Råcksta', ' Kristineberg', ' Vällingby',
       ' Abrahamsberg', ' Norra Ängby Björketorp', ' Gärdet',
       ' Beckomberga', ' Hagsätra', ' Bandhagen', ' Ham

In [27]:
# count average price per m2 for each area
print('Nr of unique areas:', len(df['area'].unique()))
df_average_price_in_area = df.groupby('area')['price_per_m2'].mean().reset_index().round(0)
df_average_price_in_area.rename(columns={'price_per_m2': 'price_per_m2_avg'}, inplace=True)
df_average_price_in_area.sort_values(by='price_per_m2_avg', ascending=False, inplace=True)
df_average_price_in_area.head(10)


Nr of unique areas: 256


,area,price_per_m2_avg
205,Södermalm-Högalid,163333.0
202,Södermalm Mosebacke,128409.0
219,Vasastan Odenplan,125631.0
140,Norr Mälarstrand,124589.0
252,Östermalm,124132.0
196,Södermalm - Reimersholme,123864.0
62,Hagastaden Vasastan,123571.0
50,Gamla Stan,122633.0
198,Södermalm / Mosebacke,122078.0
221,Vasastan-Sibirien,121639.0


In [28]:
# convert df_average_price_in_area to a dictionary
average_price_in_area = df_average_price_in_area.set_index('area').T.to_dict('list')

# apply lambda function to df to get average price per m2 for each area
df['area_price_per_m2'] = df.apply(lambda row: average_price_in_area[row['area']][0], axis=1)

In [29]:
df

,name,price,size,area,asking_price,rooms,type,price_per_m2,area_price_per_m2
0,Jämtlands­gatan 152,2600000,78.0,Hässelby-Vällingby,NaN,3.0,Lägenhet,33333.33,57544.0
1,Jämtlands­gatan 97,2800000,66.0,Vällingby Parkstad,2 795 000 kr,2.0,Lägenhet,42424.24,44364.0
2,Dala­gatan 86G,3320000,34.0,Vasastan,3 395 000 kr,1.0,Lägenhet,97647.06,113740.0
3,Valhalla­vägen 155,6500000,67.0,Östermalm Nedre Gärdet,6 200 000 kr,2.5,Lägenhet,97014.93,101733.0
4,Rådmans­gatan 1B,10850000,81.0,Östermalm,11 250 000 kr,3.0,Lägenhet,133950.62,124132.0
...,...,...,...,...,...,...,...,...,...
3633,Ymsen­vägen 8,5075000,70.0,Årsta,4 495 000 kr,3.0,Lägenhet,72500.00,63427.0
3634,Sandfjärds­gatan 98,4370000,77.0,Årsta,3 195 000 kr,4.0,Lägenhet,56753.25,63427.0
3635,Möckel­vägen 9,3200000,40.0,Årsta,2 495 000 kr,2.0,Lägenhet,80000.00,63427.0
3636,Sandfjärds­gatan 15,2900000,57.5,Årsta,2 595 000 kr,2.0,Lägenhet,50434.78,63427.0


Since we don't want to use categorical data, we can replace the area with the average selling price / m2.

In [30]:
# find unique values in area
print(len(df['area'].unique()))

# find number of nan values in rooms
print(len(df[df['rooms']=='N/A']))

# drop nan values in rooms (for now), change this later
df = df[df['rooms']!='N/A']

256
0


## Preparing for training model

In [39]:
df_x = df[['size', 'rooms', 'area_price_per_m2']]
df_y = df['price']

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

In [44]:
#XGB Regressor 
rf_regressor = RandomForestRegressor(n_estimators=20) #max_depth=15
rf_regressor.fit(x_train, y_train)
y_pred = rf_regressor.predict(x_test)

In [49]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))

Mean Absolute Error: 672577.1883001183


In [50]:
from sklearn.model_selection import cross_val_score

# use cross validation to find the best number of estimators
scores = cross_val_score(rf_regressor, x_train, y_train, cv=5)
print('Cross validation scores:', scores)
print('Mean cross validation score:', scores.mean())


Cross validation scores: [0.87623993 0.88865936 0.85743384 0.88669534 0.87625104]
Mean cross validation score: 0.8770559036044296


In [53]:
rf_regressor.predict(x_test.iloc[0])

/Users/ericjohansson/opt/miniconda3/envs/junk-filter-env/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=[5.8000e+01 2.0000e+00 9.8282e+04].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [9]:
html_page = requests.get('https://www.booli.se/bostad/4096697')
soup = BeautifulSoup(html_page.text, 'html.parser')
select = soup.find('div', class_="_2IyrD _36W0F _16dH_")

name = select.find('h1').text
price = select.find('h2').text
size_and_area = select.findAll('h4')
size = size_and_area[0].text
area = size_and_area[1].text

'44 m², 2 rum'